### 1.0 Libraries and Directories

In [1]:
import ee
import geemap
import pprint as pp

In [2]:
ee.Authenticate()
ee.Initialize()

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


In [3]:
glad = ee.ImageCollection('projects/glad/water/individualMonths')

rois = ee.data.listAssets('projects/alpod-412314/assets/ROIs/')
rois_ids = [roi['id'] for roi in rois['assets']]
rois_ids.remove('projects/alpod-412314/assets/ROIs/all_regions')
pp.pp(rois_ids)

['projects/alpod-412314/assets/ROIs/AKCP_roi',
 'projects/alpod-412314/assets/ROIs/MRD_TUK_Anderson_roi',
 'projects/alpod-412314/assets/ROIs/MRD_roi',
 'projects/alpod-412314/assets/ROIs/TUK_roi',
 'projects/alpod-412314/assets/ROIs/YKdelta_roi',
 'projects/alpod-412314/assets/ROIs/YKflats_roi',
 'projects/alpod-412314/assets/ROIs/anderson_plain_roi']


### 2.0 Combine all the GLAD observations for June & August for 20 years. 

In [4]:
years = range(1990, 2025) # 2016-2020 is the common timeframe | 1999-2020 is the full timeframe

aug_sys_idx =[f'{year}_08' for year in years]
june_sys_idx =[f'{year}_06' for year in years]

jun_imgs = glad.filter(ee.Filter.inList('system:index', june_sys_idx))
jun_max_obs = jun_imgs.size().getInfo()
aug_imgs = glad.filter(ee.Filter.inList('system:index', aug_sys_idx))
aug_max_obs = aug_imgs.size().getInfo()

monthly_datasets = [
    (jun_imgs, jun_max_obs, 'june'), 
    (aug_imgs, aug_max_obs, 'aug'),
]

print(jun_max_obs, aug_max_obs)

22 22


In [7]:
pp.pp(jun_imgs.getInfo())

{'type': 'ImageCollection',
 'bands': [],
 'version': 1647386366866962,
 'id': 'projects/glad/water/individualMonths',
 'features': [{'type': 'Image',
               'bands': [{'id': 'wp',
                          'data_type': {'type': 'PixelType',
                                        'precision': 'int',
                                        'min': 0,
                                        'max': 255},
                          'dimensions': [1432001, 560001],
                          'crs': 'EPSG:4326',
                          'crs_transform': [0.00025,
                                            0,
                                            -179,
                                            0,
                                            -0.00025,
                                            80]}],
               'version': 1612794621811240.0,
               'id': 'projects/glad/water/individualMonths/1999_06',
               'properties': {'system:time_start': 929404800000,


In [5]:
def select_invalid(img, mask_val):
    img_unmasked = img.unmask(mask_val)
    inval_pixels = img_unmasked.eq(mask_val)
    inval_pixels = inval_pixels.updateMask(ee.Image.constant(1))
    return inval_pixels

def mask_invalid(img, mask_val):
    data_mask = img.neq(mask_val)
    data = img.updateMask(data_mask)
    return data

def calc_valid_occurance(img_collection, max_obs, mask_val):
    # Sum the invalid pixels in each image
    invalid_masks = img_collection.map(lambda img: select_invalid(img, mask_val))
    sum_invalid = invalid_masks.reduce(ee.Reducer.sum())

    # Sum the valid water observations in each image
    data = img_collection.map(lambda img: mask_invalid(img, mask_val))
    sum_water = data.reduce(ee.Reducer.sum())

    # Calculate the % valid occurence
    max_obs_img = ee.Image.constant(max_obs)
    denom_img = max_obs_img.subtract(sum_invalid)
    valid_occurence = sum_water.divide(denom_img)

    return valid_occurence



target_imgs = []

for ds in monthly_datasets:
    valid_occurance = calc_valid_occurance(ds[0], ds[1], 255)
    month = ds[2]
    target_imgs.append((valid_occurance, month))

In [11]:
june = target_imgs[0][0]


In [12]:
Map = geemap.Map()
viz_params = {
    'min': 0,
    'max': 22,  
    'palette': ['red', 'yellow', 'blue']
}
Map.addLayer(june, viz_params, 'Summed Image')

Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [11]:
def export_rois(roi_id, target_img, month):
    
    roi_name_temp = roi_id.split('/')[-1]
    roi_name = roi_name_temp.split('_roi')[0]
    feature = ee.FeatureCollection(roi_id).first()
    bounds = feature.geometry()

    clipped_img = target_img.clip(bounds)

    export_params = {
        'image': clipped_img,
        'description': f'gladFullMonthly_{roi_name}_month_{month}_', 
        'fileNamePrefix': f'gladFullMonthly_{roi_name}_month_{month}_',
        'folder': 'glad_exports', 
        'scale': 30,
        'crs': 'EPSG:4326',
        'region': bounds,
        'fileFormat': 'GeoTIFF',
        'maxPixels': 1e13
    }

    ee.batch.Export.image.toDrive(**export_params).start()

In [12]:
print(target_imgs)

[(<ee.image.Image object at 0x158db2840>, 'june'), (<ee.image.Image object at 0x158db3ce0>, 'aug')]


In [13]:
for roi_id in rois_ids:
    for img in target_imgs:
        image = img[0]
        month = img[1]
        export_rois(roi_id, image, month)